# Linear Regression
The purpose of this notebook is to practice training (also known as fitting), interpreting and evaluating linear regression predictive models. 
We will use Python packages: pandas, matplotlib and scikit-learn.
Besides the material presented in this notebook, please also read this [notebook](http://www.dataschool.io/linear-regression-in-python/) that is very well written, contains many useful details and gives pointers to further reading. 

Training a linear regression model means estimating a set of weights (one weight per feature, plus an extra weight called the bias or the intercept) on a dataset called the training set. 

The model estimated is a linear model taking the form:

$target\_feature = w_0 + w_1 * feature_1 + w_2*feature_2 + ...+ w_n*feature_n $

The learned model can be used to predict the target feature for new examples where we know the descriptive features, but not the target feature. This is called the test example or the test data. In this notebook we will see the difference between evaluating the model on the training data and measuring the model error (called in-sample error) versus evaluating the model on the test data and measuring that error (called out-of-sample error). It is recommended that we always evaluate our model on a second data sample that was not used during training. This way we avoid overfitting or memorising the training data.

## Reading data

In [86]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

In [87]:
# Read a CSV dataset with 10 example offices into a dataframe.
# The data is described by 5 features (4 descriptive features: Size, Floor, BroadbandRate, EnergyRating;
# the target feature: RentalPrice).


# Read csv file into a dataframe.
df = pd.read_csv('Offices.csv')
df.head(10)

,ID,Size,Floor,BroadbandRate,EnergyRating,RentalPrice
0,1,500,4,8,C,320
1,2,550,7,50,A,380
2,3,620,9,7,A,400
3,4,630,5,24,B,390
4,5,665,8,100,C,385
5,6,700,4,8,B,410
6,7,770,10,7,B,480
7,8,880,12,50,A,600
8,9,920,14,8,C,570
9,10,1000,9,24,B,620


In [88]:
# Print the average RentalPrice in our dataset.
# We could use this as a very simple baseline prediction model.
# A better prediction model should at least improve on this baseline model.
df.RentalPrice.mean()

455.5

In [89]:
# Print the feature types in our dataset.
df.dtypes

ID                int64
Size              int64
Floor             int64
BroadbandRate     int64
EnergyRating     object
RentalPrice       int64
dtype: object

### Preparing the data

In [90]:
# Prepare the descriptive features
print(df.head(10))
cont_features = ['Size']
#cont_features = ['Size', 'Floor', 'BroadbandRate']

X = df[cont_features]
y = df.RentalPrice

print("\nDescriptive features in X:\n", X)
print("\nTarget feature in y:\n", y)

   ID  Size  Floor  BroadbandRate EnergyRating  RentalPrice
0   1   500      4              8            C          320
1   2   550      7             50            A          380
2   3   620      9              7            A          400
3   4   630      5             24            B          390
4   5   665      8            100            C          385
5   6   700      4              8            B          410
6   7   770     10              7            B          480
7   8   880     12             50            A          600
8   9   920     14              8            C          570
9  10  1000      9             24            B          620

Descriptive features in X:
    Size
0   500
1   550
2   620
3   630
4   665
5   700
6   770
7   880
8   920
9  1000

Target feature in y:
 0    320
1    380
2    400
3    390
4    385
5    410
6    480
7    600
8    570
9    620
Name: RentalPrice, dtype: int64


## Multiple linear regression (using more than one feature)
### Training the model

In [91]:
# Use more features for training
# Train aka fit, a model using all continuous features.

multiple_linreg = LinearRegression().fit(X[cont_features], y)

# Print the weights learned for each feature.
print("Features: \n", cont_features)
print("Coeficients: \n", multiple_linreg.coef_)
print("\nIntercept: \n", multiple_linreg.intercept_)

Features: 
 ['Size']
Coeficients: 
 [0.62064008]

Intercept: 
 6.466899807301957


### Testing the model
Using the trained model to predict the target feature RentalPrice, given the descriptive features Size, Floor  BroadbandRate.

In [92]:
multiple_linreg_predictions = multiple_linreg.predict(X[cont_features])

print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([y, pd.DataFrame(multiple_linreg_predictions, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg)


Predictions with multiple linear regression: 

   RentalPrice   Predicted
0          320  316.786941
1          380  347.818946
2          400  391.263751
3          390  397.470152
4          385  419.192555
5          410  440.914958
6          480  484.359764
7          600  552.630173
8          570  577.455776
9          620  627.106983


In [93]:
#Pair the actual and the predicted values
#This can be done directly with sklearn functions, but below is a manual example to understand how it works
prediction_errors = y - multiple_linreg_predictions
print("Actual - Predicted:\n", prediction_errors)
print("\n(Actual - Predicted) squared:\n", prediction_errors**2)

Actual - Predicted:
 0     3.213059
1    32.181054
2     8.736249
3    -7.470152
4   -34.192555
5   -30.914958
6    -4.359764
7    47.369827
8    -7.455776
9    -7.106983
Name: RentalPrice, dtype: float64

(Actual - Predicted) squared:
 0      10.323746
1    1035.620265
2      76.322040
3      55.803174
4    1169.130827
5     955.734631
6      19.007541
7    2243.900508
8      55.588601
9      50.509207
Name: RentalPrice, dtype: float64


In [94]:
# Print the Mean Squared Error of the model on the training set
mse = (prediction_errors** 2).mean()
rmse = ((prediction_errors** 2).mean())**0.5

print("\nMean Squared Error:\n", mse)
print("\nRoot Mean Squared Error:\n", rmse)


Mean Squared Error:
 567.19405389423

Root Mean Squared Error:
 23.815836199769052


In [95]:
print("|Actual - Predicted|:\n", abs(prediction_errors))

|Actual - Predicted|:
 0     3.213059
1    32.181054
2     8.736249
3     7.470152
4    34.192555
5    30.914958
6     4.359764
7    47.369827
8     7.455776
9     7.106983
Name: RentalPrice, dtype: float64


In [96]:
# Print the Mean Absolute Error of the model on the training set
mae = abs(prediction_errors).mean()
print("\nMean Absolute Error:\n", mae)


Mean Absolute Error:
 18.30003772392207


In [97]:
# Print the R2 of the model on the training set
#Pair the actual and the predicted values
#This can be done directly with sklearn functions, but below is a manual example to understand how it works

prediction_errors = y - multiple_linreg_predictions
print("Actual - Predicted:\n", prediction_errors)
print("\n(Actual - Predicted) squared:\n", prediction_errors**2)
print("\n Sum of squared errors:\n", sum(prediction_errors**2))

avg_predictions =np.ones(y.shape[0])* df.RentalPrice.mean()

print("\nAverageModelPredictions:\n", avg_predictions)
avgpredictions_errors = y - avg_predictions
print("Actual - AvgPredictions:\n", avgpredictions_errors)
print("\n(Actual - AvgPredictions) squared:\n", avgpredictions_errors**2)
print("\n Total sum of squared errors:\n", sum(avgpredictions_errors**2))

r2 = 1 - sum(prediction_errors**2)/sum(avgpredictions_errors**2)
print("\n R2:\n", r2)


Actual - Predicted:
 0     3.213059
1    32.181054
2     8.736249
3    -7.470152
4   -34.192555
5   -30.914958
6    -4.359764
7    47.369827
8    -7.455776
9    -7.106983
Name: RentalPrice, dtype: float64

(Actual - Predicted) squared:
 0      10.323746
1    1035.620265
2      76.322040
3      55.803174
4    1169.130827
5     955.734631
6      19.007541
7    2243.900508
8      55.588601
9      50.509207
Name: RentalPrice, dtype: float64

 Sum of squared errors:
 5671.9405389423

AverageModelPredictions:
 [455.5 455.5 455.5 455.5 455.5 455.5 455.5 455.5 455.5 455.5]
Actual - AvgPredictions:
 0   -135.5
1    -75.5
2    -55.5
3    -65.5
4    -70.5
5    -45.5
6     24.5
7    144.5
8    114.5
9    164.5
Name: RentalPrice, dtype: float64

(Actual - AvgPredictions) squared:
 0    18360.25
1     5700.25
2     3080.25
3     4290.25
4     4970.25
5     2070.25
6      600.25
7    20880.25
8    13110.25
9    27060.25
Name: RentalPrice, dtype: float64

 Total sum of squared errors:
 100122.5

 R2:


In [98]:
#This function is used repeatedly to compute all metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))
        

In [99]:
printMetrics(y, multiple_linreg_predictions)


MAE:  18.30003772392207
RMSE:  23.815836199769052
R2:  0.9433499908717591


# Training with continuous and categorical features

In [100]:
# Use more features for training
# Train aka fit, a model using all continuous and categorical features.
EnergyRating_dummies = pd.get_dummies(df['EnergyRating'], prefix='EnergyRating', drop_first=True)
print("EnergyRatingDummies:", EnergyRating_dummies)

categ_features = EnergyRating_dummies.columns.values.tolist()

features = cont_features + categ_features
print("\nCont features: ", cont_features)
print("Categ features: ", categ_features)
print("Features: ", features)

EnergyRatingDummies:    EnergyRating_B  EnergyRating_C
0               0               1
1               0               0
2               0               0
3               1               0
4               0               1
5               1               0
6               1               0
7               0               0
8               0               1
9               1               0

Cont features:  ['Size']
Categ features:  ['EnergyRating_B', 'EnergyRating_C']
Features:  ['Size', 'EnergyRating_B', 'EnergyRating_C']


In [101]:
df_all = pd.concat([df, EnergyRating_dummies], axis=1)
print(df_all)

df_all = df_all.drop('EnergyRating', axis = 1)
print(df_all)

   ID  Size  Floor  BroadbandRate EnergyRating  RentalPrice  EnergyRating_B  \
0   1   500      4              8            C          320               0   
1   2   550      7             50            A          380               0   
2   3   620      9              7            A          400               0   
3   4   630      5             24            B          390               1   
4   5   665      8            100            C          385               0   
5   6   700      4              8            B          410               1   
6   7   770     10              7            B          480               1   
7   8   880     12             50            A          600               0   
8   9   920     14              8            C          570               0   
9  10  1000      9             24            B          620               1   

   EnergyRating_C  
0               1  
1               0  
2               0  
3               0  
4               1  
5         

In [102]:
#We can also do this directly for all categorical features
df = pd.get_dummies(df, drop_first=True)
df

,ID,Size,Floor,BroadbandRate,RentalPrice,EnergyRating_B,EnergyRating_C
0,1,500,4,8,320,0,1
1,2,550,7,50,380,0,0
2,3,620,9,7,400,0,0
3,4,630,5,24,390,1,0
4,5,665,8,100,385,0,1
5,6,700,4,8,410,1,0
6,7,770,10,7,480,1,0
7,8,880,12,50,600,0,0
8,9,920,14,8,570,0,1
9,10,1000,9,24,620,1,0


In [103]:
X = df_all[features]
y = df_all.RentalPrice

print("\nDescriptive features in X:\n", X)
print("\nTarget feature in y:\n", y)


Descriptive features in X:
    Size  EnergyRating_B  EnergyRating_C
0   500               0               1
1   550               0               0
2   620               0               0
3   630               1               0
4   665               0               1
5   700               1               0
6   770               1               0
7   880               0               0
8   920               0               1
9  1000               1               0

Target feature in y:
 0    320
1    380
2    400
3    390
4    385
5    410
6    480
7    600
8    570
9    620
Name: RentalPrice, dtype: int64


In [104]:
# Use more features for training
# Train aka fit, a model using all continuous and categorical features.

multiple_linreg = LinearRegression().fit(X, y)

# Print the weights learned for each feature.
#print("Features: \n", features)
#print("Coeficients: \n", multiple_linreg.coef_)
print("\nIntercept: \n", multiple_linreg.intercept_)
print("Features and coeficients:", list(zip(features, multiple_linreg.coef_)))


Intercept: 
 20.810909890754317
Features and coeficients: [('Size', 0.6427157416232864), ('EnergyRating_B', -43.91560964880125), ('EnergyRating_C', -42.49835031893834)]


In [105]:
multiple_linreg_predictions = multiple_linreg.predict(X[features])

print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([df, pd.DataFrame(multiple_linreg_predictions, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg)


Predictions with multiple linear regression: 

   ID  Size  Floor  BroadbandRate  RentalPrice  EnergyRating_B  \
0   1   500      4              8          320               0   
1   2   550      7             50          380               0   
2   3   620      9              7          400               0   
3   4   630      5             24          390               1   
4   5   665      8            100          385               0   
5   6   700      4              8          410               1   
6   7   770     10              7          480               1   
7   8   880     12             50          600               0   
8   9   920     14              8          570               0   
9  10  1000      9             24          620               1   

   EnergyRating_C   Predicted  
0               1  299.670430  
1               0  374.304568  
2               0  419.294670  
3               0  381.806217  
4               1  405.718528  
5               0  426.796319  
6

In [106]:
printMetrics(y, multiple_linreg_predictions)


MAE:  11.361903365349338
RMSE:  13.598640185098985
R2:  0.9815303238648866


# Evaluation with train/test split

In [107]:
# Split the data into train and test sets
# Take a third (random) data samples as test data, rest as training data
# Note that this training set if very small and the model will not be very reliable due to this sample size problem.
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Training data:\n", pd.concat([X_train, y_train], axis=1))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

Training data:
    Size  EnergyRating_B  EnergyRating_C  RentalPrice
4   665               0               1          385
1   550               0               0          380
3   630               1               0          390
6   770               1               0          480
2   620               0               0          400
7   880               0               0          600
0   500               0               1          320

Test data:
    Size  EnergyRating_B  EnergyRating_C  RentalPrice
9  1000               1               0          620
5   700               1               0          410
8   920               0               1          570


In [108]:
# Train on the training sample and test on the test sample.
linreg = LinearRegression().fit(X_train, y_train)
# Print the weights learned for each feature.
#print(linreg_train.coef_)
print("Features and coeficients:", list(zip(features, linreg.coef_)))

Features and coeficients: [('Size', 0.6397595747851571), ('EnergyRating_B', -35.66265957975263), ('EnergyRating_C', -42.990909542496645)]


In [109]:
# Predicted price on training set
train_predictions = linreg.predict(X_train)
print("Actual values of training:\n", y_train)
print("Predictions on training:", train_predictions)
printMetrics(y_train, train_predictions)



Actual values of training:
 4    385
1    380
3    390
6    480
2    400
7    600
0    320
Name: RentalPrice, dtype: int64
Predictions on training: [405.28016492 374.69872336 390.21682977 479.78317023 419.4818936
 585.81938304 299.71983508]

MAE:  11.42253950907274
RMSE:  14.299859230533412
R2:  0.9714250314369299


In [110]:
# Predicted price on test set
test_predictions = linreg.predict(X_test)
print("Actual values of test:\n", y_test)
print("Predictions on test:", test_predictions)
printMetrics(y_test, test_predictions)

Actual values of test:
 9    620
5    410
8    570
Name: RentalPrice, dtype: int64
Predictions on test: [626.92787244 435.         568.41885649]

MAE:  11.169671981852218
RMSE:  15.005503782306645
R2:  0.9719323227998979


# Evaluation with cross-validation

In [111]:
sorted(metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [114]:
scores = -cross_val_score(LinearRegression(), X, y, scoring='neg_mean_absolute_error', cv=5)
scores

array([31.66811733, 19.87021387, 27.60324484, 19.04539189,  1.4933022 ])

In [115]:
metrics = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
scores = cross_validate(LinearRegression(), X, y, scoring=metrics, cv=5)
scores

/Users/georgianaifrim/anaconda3/envs/comp47350py37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_neg_mean_absolute_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/georgianaifrim/anaconda3/envs/comp47350py37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_neg_mean_squared_error'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/georgianaifrim/anaconda3/envs/comp47350py37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_r2'), which will not be available by default any more in 0.21. If you need training scores, please set ret

{'fit_time': array([0.00271511, 0.00192618, 0.00204802, 0.00309086, 0.00214481]),
 'score_time': array([0.00452018, 0.00295997, 0.00322819, 0.00270915, 0.00251508]),
 'test_neg_mean_absolute_error': array([-31.66811733, -19.87021387, -27.60324484, -19.04539189,
         -1.4933022 ]),
 'train_neg_mean_absolute_error': array([ -9.52606841,  -9.78634874,  -7.58251958, -12.23685506,
        -14.1071813 ]),
 'test_neg_mean_squared_error': array([-1178.62999792,  -479.35721708,  -778.03978181,  -430.78620284,
           -2.23766757]),
 'train_neg_mean_squared_error': array([-103.88197406, -150.12073129,  -99.74506663, -173.67073651,
        -231.00856059]),
 'test_r2': array([ -0.30958889, -18.17428868,  -3.9794546 ,   0.88033717,
          0.99641973]),
 'train_r2': array([0.98821145, 0.98679123, 0.99126958, 0.98149286, 0.96320192])}

In [116]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_neg_mean_absolute_error',
 'test_neg_mean_squared_error',
 'test_r2',
 'train_neg_mean_absolute_error',
 'train_neg_mean_squared_error',
 'train_r2']